In [1]:
from datetime import time
import pyspark
from pyspark.sql import SQLContext
import dateutil.parser
import mysql.connector


In [2]:
sparkCont = pyspark.SparkContext('local[*]')
sc = SQLContext(sparkCont)

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/08 08:19:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/ubuntu/cs179g/lib/python3.8/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
releases = sc.read.option("multiline", "true").json("./CS179G_Project/cs179g_crawler/releases.json")
releases.createOrReplaceTempView("releases_json")
df = sc.sql("SELECT * from releases_json")
releases_info = df.select('tag', 'features_and_fixes', 'pull_request_ids', 'date').collect()

In [4]:
pull_requests = sc.read.option("multiline", "true").json("./CS179G_Project/pull_requests_filtered.json")
pull_requests.createOrReplaceTempView("pull_requests_json")
df = sc.sql("SELECT * from pull_requests_json")
pull_requests_info = df.select('id', 'linked_issue', 'title').collect() #need to merge this with linked_issues, and issue_times

issues = sc.read.option("multiline", "true").json("./CS179G_Project/issues_filtered.json")
issues.createOrReplaceTempView("issues_json")
df = sc.sql("SELECT * from issues_json")
issues_info = df.select('title', 'url', 'date', 'status').collect() #need to merge this with other issue fields

In [136]:
#make the table
db_connection = mysql.connector.connect(user="ckong", password="cs179g")
db_cursor = db_connection.cursor(buffered=True)
db_cursor.execute("USE cs179g;")
db_cursor.execute("CREATE TABLE IF NOT EXISTS TimeDifferences(\
    issue_titles TEXT, \
    issue_statuses TEXT, \
    release_features_and_fixes TEXT, \
    time_differences FLOAT,\
    release_tags TEXT);")

In [140]:
#reset the table
db_cursor.execute("DROP TABLE TimeDifferences;")

In [141]:
db_cursor.execute("SELECT * FROM TimeDifferences;")
records = db_cursor.fetchall()
# print(db_cursor.fetchall())

for row in records:
        print("issue_titles = ", row[0])
        print("issue_statuses = ", row[1])
        print("release_features_and_fixes = ", row[2])
        print("time_differences  = ", row[3])
        print("release_tags  = ", row[4], "\n")

ProgrammingError: 1146 (42S02): Table 'cs179g.TimeDifferences' doesn't exist

In [137]:
db_cursor.execute("SELECT * FROM TimeDifferences;")
records = db_cursor.fetchall()

In [132]:
# testing mysql commands
row = {}
row['issue_titles'] = 'test'
row['issue_statuses'] = 'test'
row['release_features_and_fixes'] = 'test'
row['time_differences'] = 9.0
row["release_tags"] = 'test'
    
query = "INSERT INTO TimeDifferences VALUES (%s, %s, %s, %s, %s);"
data = (row['issue_titles'], row['issue_statuses'], row['release_features_and_fixes'], row['time_differences'], row['release_tags'])

db_cursor.execute(query, data)
# db_cursor.execute("INSERT INTO TimeDifferences (issue_titles, release_features_and_fixes, time_differences) VALUES (row['issue_titles'], 'test', 9.0);")

In [139]:
row = { "issue_titles": [], "issue_statuses": [], "release_features_and_fixes": [], "time_differences": [], "release_tags": [] }

for request in pull_requests_info: #loop through pull requests
    issueDate = ""
    releaseDate = ""
    timeDifference = 0.0
    issue_title = ""
    issue_status = ""
    for issue in issues_info: #loop through issues
        if(len(request.linked_issue) > 1): #check if linked issue is not empty
            if(issue.title == request.linked_issue[1]): #if it is not empty, check if the issue title matches one of the linked issues 
                issueDate = dateutil.parser.parse(issue.date).timestamp()
                issue_title = issue.title
                issue_status = issue.status
                # print(issueDate)
                break #leave issue for loop. we are done checking for issues until next pull request. 
    if(issueDate): #if we had a matching issue above, loop through releases to get release date
        for release in releases_info:
            if(len(request.id) > 0):
                if(request.id[0] in release.pull_request_ids):
                    releaseDate = dateutil.parser.parse(release.date).timestamp()
                    timeDifference = releaseDate - issueDate
                    
                    row['issue_titles'] = issue_title
                    row['issue_statuses'] = issue_status
                    row['issue_statuses'] = '' # need to concatenate issue_statuses indices to store it as 1 string
                    for i in range(len(issue_status)):
                        row['issue_statuses'] += ' ' + issue_status[i]
                    
                    row['release_features_and_fixes'] = '' # need to concatenate features_and_fixes indices to store it as 1 string
                    for i in range(len(release.features_and_fixes)):
                        row['release_features_and_fixes'] += release.features_and_fixes[i]
                        
                    row['time_differences'] = timeDifference
                    row['release_tags'] = release.tag
                                        
                    query = "INSERT INTO TimeDifferences VALUES (%s, %s, %s, %s, %s);"
                    data = (row['issue_titles'], row['issue_statuses'], row['release_features_and_fixes'], row['time_differences'], row['release_tags'])

                    db_cursor.execute(query, data)
                    db_cursor.execute("FLUSH TABLES;")
                    break #leave release for loop. we are done checking for releases until next pull request. we have our time difference.

print('Finished inserting data into MySQL')
                    
db_cursor.execute("SELECT * FROM TimeDifferences;")
records = db_cursor.fetchall()
# print(db_cursor.fetchall())

for record in records:
        print("issue_titles = ", record[0])
        print("issue_statuses = ", record[1])
        print("release_features_and_fixes = ", record[2])
        print("time_differences  = ", record[3])
        print("release_tags  = ", record[4], "\n")

Finished inserting data into MySQL
issue_titles =  Changingplaceholderon<input>causesonChangeinIE
issue_statuses =  
release_features_and_fixes =   Previously we would generate a large string of HTML and then set . At the time, this was decided to be faster than using  for the majority of cases and browsers that we supported. Browsers have continued to improve and so overwhelmingly this is no longer true. By using  we can make other parts of React faster. ( in ) As a result of using , we can prime the node cache as we create DOM nodes, allowing us to skip a potential lookup (which used the  attribute). Root nodes will have a  attribute and server generated markup will still contain . ( in ) ReactDOM will now render plain text nodes interspersed with comment nodes that are used for demarcation. This gives us the same ability to update individual pieces of text, without creating extra nested nodes. If you were targeting these s in your CSS, you will need to adjust accordingly. You can al